# 年度业绩报告总体分析

In [1]:
import pandas as pd
import akshare as ak
import altair as alt
import time

In [2]:
# this func will get performance report of all listed companys
def getdata() :
    # 创建一个基数列表，注意列表是不包含最后一个年份的。
    l1 = list(range(20100000, 20230000, 10000))
    l2 = [331, 630, 930, 1231]
    qs = [ m+n for m in l1 for n in l2]
    yjbg = pd.DataFrame()
    for q in qs:
        df = ak.stock_yjbb_em(date=str(q))
        df['qs'] = q # add a new col of period
        yjbg = yjbg.append(df, ignore_index=True)
        time.sleep(2)
        print(q)
    yjbg.to_csv('yjbg2022.csv', index=False)

    return

In [8]:
df = pd.read_pickle('../../yjbg2022.pkl.gz')

## 数据集

利用`akshare`的业绩报告模块，下载全部上市公司自2010年起的业绩报告。

业绩报告包含的信息：

In [42]:
col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
col_profile.columns = ['列名', '类型', '缺失比%','样例']
col_profile

,列名,类型,缺失比%,样例
0,序号,int64,0.000000,4116
1,股票代码,int64,0.000000,870231
2,股票简称,object,0.000000,源悦汽车
3,每股收益,float64,1.772486,-0.08
4,营业收入-营业收入,float64,0.188940,71771743.91
5,营业收入-同比增长,float64,9.823344,-33.078996
6,营业收入-季度环比增长,float64,38.700052,20.4432
7,净利润-净利润,float64,0.026502,-5532474.8
8,净利润-同比增长,float64,9.352240,-188.331925
9,净利润-季度环比增长,float64,38.399182,-518.5794


初步分析此数据集，有如下观点：

- 共17列，其中序号无意义，代码数据格式不正确，但可以不考虑。
- 最后一列时报告期，检视数据时发现2010年的报告期和公告日期有矛盾，但数据集尾部数据正常。
- 净利润和营业收入的缺失值极少，应该不影响大局。
- 净资产收益率缺乏率2.5%，考虑使用。